In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
import bets
import sqlite3
import os
import glob
import datetime as dt
import time
#import ot
import json
import requests
import statsmodels.api as sm 
from tqdm import tqdm
pd.set_option('display.float_format', lambda x: '%.3f' % x)

nba = bets.nba()
import seaborn as sns
import matplotlib.pyplot as plt
import re
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
from nba_api.stats.endpoints import BoxScoreAdvancedV3,leaguegamefinder,BoxScoreSummaryV2,PlayByPlayV2,LeagueDashOppPtShot,PlayerProfileV2,GameRotation,LeagueDashPlayerShotLocations

In [3]:
def runDay(date,toDb = False,useNew = False):
    yst = (pd.to_datetime(today) + pd.to_timedelta(-1,unit='day')).strftime(format='%Y-%m-%d')
    if useNew:
        data =  nba.threeData() #there will need to be an edit to only get data for today
        model = pd.read_pickle('data/model/ThrMod2.pkl')
    else:
        data =  nba.threeData() #there will need to be an edit to only get data for today
        model = pd.read_pickle('data/model/upThrLog.pkl')
    #create model data and scale
    X = nba.cleanNaThr(data)
    X = nba.scaleData(X)
    idx = data[data.game_date==date].index
    #make day's predictions
    preds = model.predict(X.loc[idx].filter(model.params.index))
    upload =  data.loc[idx][['game_date','game_id','player_id','name','team']].join(preds)
    preds = preds.join(data[['name']])
    if len(upload) <19:
        for i in range(1,(19 - len(upload))+1):
            upload['add_{}'.format(i)] = 0.0000

    # if toDb:
    #     nba.conn.execute("DELETE FROM preds where game_date = '{}' ".format(yst))
    #     nba.conn.commit()
    #     try:
    #         nba.insert_data(upload,'preds')
    #         print('updated prediction table')
    #     except:
    #         print('PREDS NOT SAVED')
    #         return upload,preds,model

    #create a data frame of the predictions
    #df = nba.probDf(preds.values,y.loc[valInd],data[['name','game_date']],valInd)
    
    #convert into odds
    c = [10,9,8,7,6,5,4,3,2,1,0]
    finalo = pd.DataFrame(np.array([nba.convertPercentToOdds(v) for r in preds[c].cumsum(axis=1).values for v in r]).reshape(preds[c].shape),
                         columns=c,index=preds.index)
    finalu = pd.DataFrame(np.array([nba.convertPercentToOdds(v) for r in preds.iloc[:,:-1].cumsum(axis=1).values for v in r]).reshape(preds[sorted(c)].shape),
                          columns = c,index=preds.index)
    overs = data[data.game_date==date][['name','team','game_id']].join(finalo.filter(preds.columns))
    unders = data[data.game_date==date][['name','team','game_id']].join(finalu.filter(preds.columns))
    return overs,unders,preds,model,data[data.game_date==date]

In [ ]:
etl.curr.execute(

### This notebook is to just run the current model and get our predictions

In [244]:
today = dt.datetime.today().strftime(format='%Y-%m-%d')
yst = (pd.to_datetime(today) + pd.to_timedelta(-1,unit='day')).strftime(format='%Y-%m-%d')
gids = nba.get_games(yst,yst)

In [245]:
print('Updating for {}'.format(yst))
nba.update_player_log([yst])
time.sleep(np.random.randint(5,15))
try:
    nba.update_shots_allowed([yst])
except:
    print('shots not updated yet')
time.sleep(np.random.randint(5,15)) 
nba.update_teamLog(gids.GAME_ID.unique())

Updating for 2025-04-13
started player logs at 06:38
getting first buckets : at 06:38


100%|███████████████████████████████████████████| 15/15 [00:38<00:00,  2.55s/it]


	completed at 06:39
started rebounds at 06:39
ended rebounds at 06:39
starting advanced box at 06:39


100%|███████████████████████████████████████████| 15/15 [00:36<00:00,  2.47s/it]


completed adv box at 06:40
start player shots at 06:40


...

Completed spot data


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]

2025-04-13 no distance from shooter data
Completed open shot data
shots not updated yet



100%|███████████████████████████████████████████| 15/15 [00:15<00:00,  1.02s/it]

teamLog has been updated with 30 rows


In [239]:
def oddsData(today,tomorrow):
    '''ISO Formatted dates for today and tomorrow returns the games that will be played today ids for odds pulls
    Inputs: isoformatted dates for today and tomorrow
    Output: list of game ids
    '''
    dct = json.loads(open('config.json','r').read())
    eventURL = 'https://api.the-odds-api.com/v4/sports/basketball_nba/events?apiKey={}&dateFormat=iso&commenceTimeFrom={}&commenceTimeTo={}'
    fr = dct.get('oddsApi').get('free')
    pd = dct.get('oddsApi').get('paid')
    if int(requests.get(eventURL.format(fr,today,tomorrow)).headers['x-requests-used'])>=490:
        print('Free is out')
        r = requests.get(eventURL.format(pd,today,tomorrow))
        print(r.headers)
        key = pd
    else:
        r= requests.get(eventURL.format(fr,today,tomorrow))
        print('Free:',r.headers)
        key = fr
    
    return [d['id'] for d in r.json()],key

In [240]:
todayIso = (dt.datetime.now()).strftime('%Y-%m-%dT%H:%M:00Z')
tomor = (dt.datetime.now() + dt.timedelta(1)).strftime('%Y-%m-%dT%H:%M:00z')
events,akey = oddsData(todayIso,tomor)

Free: {'Date': 'Sun, 13 Apr 2025 10:41:57 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '862', 'Connection': 'keep-alive', 'X-Requests-Last': '0', 'vary': 'Accept-Encoding', 'content-encoding': 'gzip', 'X-Requests-Used': '270', 'X-Requests-Remaining': '230', 'Apigw-Requestid': 'I9Q2fjgWIAMEahg='}


In [241]:
url = 'https://api.the-odds-api.com/v4/sports/basketball_nba/events/{}/odds?apiKey={}&regions=us&markets=player_threes,player_threes_alternate&dateFormat=iso&oddsFormat=american&bookmakers=draftkings%2Cfanduel%2cbetmgm'

In [242]:
df = pd.DataFrame()
for event in events:
    r = requests.get(url.format(event,akey))
    game = r.json()
    for key in game.get('bookmakers'):
        bk = key.get('title')
        for mrkt in key.get('markets'):
            temp = pd.DataFrame(mrkt.get('outcomes'))
            temp.columns = ['over_under','name','price','threesMade']
            temp['book'] = bk
            temp['mrkt'] = mrkt
            df = pd.concat([temp,df])   
odds = df.pivot_table(index=['name','threesMade','over_under'],columns=['book']).reset_index()
odds.columns = [col[1] if col[1]!= '' else col[0] for col in odds.columns]
#Name changes
odds.name = np.where(odds.name=='Herb Jones','Herbert Jones',odds.name)
odds.name = np.where(odds.name =='Alex Sarr','Alexandre Sarr',odds.name)
odds.name = np.where(odds.name.str.contains('\.'),odds.name.str.replace('.',''),odds.name)
#over.name = np.where(over.name.str.contains('\.'),over.name.str.replace('.',''),over.name)

In [243]:
odds.to_csv('data/csv/{}odds.csv'.format(today),index=False)

In [85]:
def check_table(minDate=None):
    '''
    Go through table and check the following:
    NA count by column
    number of instances by team(gp for shotsAllowed and teamLog, number of players with stats in the time period for plyrlog
    will check the following columns for each table against nba api:
    teamLog: gp, home, win, q1_pts
    shotsAllowed: gp,  ra_fga, wide_fg3a, drb0_fg2a
    plyrLog: gp, min, tam_first, dreb, ra_fga,starter
    Date will default to beginning of current season
    '''
    yst = (dt.datetime.today() + dt.timedelta(-1)).strftime('%Y-%m-%d')
    date = minDate if minDate!=None else "2024-10-01"##nba.curSeasonStart replace when next instanstansioation occurs
    tl = pd.read_sql('''SELECT teamAbrv, teamLog.*
                    FROM teamLog
                    LEFT join teams using(team_id)
                    WHERE game_date BETWEEN "{}" AND "{}"
                    '''.format(date,yst)
                     ,nba.conn)

In [ ]:
over,under,preds,model,data = runDay(today,True)

In [211]:
odds[odds.name.str.contains('Green')].head(50)

,name,threesMade,over_under,DraftKings,ESPN BET,FanDuel
0,AJ Green,1.500,Over,-191.500,-160.000,NaN
1,AJ Green,1.500,Under,145.000,120.000,NaN
2,AJ Green,2.500,Over,145.000,NaN,NaN
3,AJ Green,3.500,Over,370.000,NaN,NaN
4,AJ Green,4.500,Over,850.000,NaN,NaN
126,Draymond Green,0.500,Over,-525.000,NaN,NaN
127,Draymond Green,1.500,Over,-115.000,-110.000,NaN
128,Draymond Green,1.500,Under,-115.000,-120.000,NaN
129,Draymond Green,2.500,Over,255.000,NaN,NaN
130,Draymond Green,3.500,Over,650.000,NaN,NaN


In [117]:
over = over.melt(id_vars=['name','team','game_id',],var_name='threesMade')
over.threesMade = over.threesMade -.5
final = odds.merge(over[['name','team','threesMade','value']],how='left',on=['name','threesMade'])
final = final[final.over_under=='Over']
final['prob'] = np.where(final.value<0, abs(final.value) / (abs(final.value) + 100), 100/(final.value +100))
final['DKKelly'] = [nba.kellyCrit(p,odd) / 8 for p,odd in zip(final.prob,final.DraftKings)]
final['FanDuelKelly'] = [nba.kellyCrit(p,odd) / 8 for p,odd in zip(final.prob,final.FanDuel)]
final['espnKelly'] = [nba.kellyCrit(p,odd) / 8 for p,odd in zip(final.prob,final['ESPN BET'])]
final['dkBet'] = [x * 100 * 5 for x in final.DKKelly.values]
final['fdBet'] = [x * 100 * 5 for x in final.FanDuelKelly.values]
final['espnBet'] = [x * 100 * 5 for x in final.espnKelly]

NameError: name 'over' is not defined

In [45]:
final

,name,threesMade,over_under,DraftKings,ESPN BET,FanDuel,team,value,prob,DKKelly,FanDuelKelly,espnKelly,dkBet,fdBet,espnBet
0,Aaron Gordon,0.500,Over,-280.000,-250.000,NaN,DEN,718.000,0.122,-0.292,NaN,-0.259,-145.966,NaN,-129.508
2,Aaron Gordon,1.500,Over,162.500,NaN,NaN,DEN,5114.000,0.019,-0.073,NaN,NaN,-36.525,NaN,NaN
4,Aaron Gordon,2.500,Over,550.000,NaN,NaN,DEN,9900.000,0.010,-0.021,NaN,NaN,-10.625,NaN,NaN
5,Ayo Dosunmu,0.500,Over,-500.000,NaN,-450.000,CHI,814.000,0.109,-0.543,-0.487,NaN,-271.472,-243.641,NaN
6,Ayo Dosunmu,1.500,Over,-110.000,-125.000,-115.000,CHI,6323.000,0.016,-0.133,-0.140,-0.152,-66.707,-69.783,-75.936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Zach LaVine,3.500,Over,130.000,130.000,125.000,CHI,9900.000,0.010,-0.094,-0.098,-0.094,-46.971,-48.875,-46.971
299,Zach LaVine,4.500,Over,280.000,NaN,250.000,CHI,9900.000,0.010,-0.043,-0.048,NaN,-21.473,-24.125,NaN
300,Zach LaVine,5.500,Over,600.000,NaN,500.000,CHI,9900.000,0.010,-0.019,-0.024,NaN,-9.688,-11.750,NaN
301,Zach LaVine,6.500,Over,NaN,NaN,1000.000,CHI,9900.000,0.010,NaN,-0.011,NaN,NaN,-5.562,NaN


In [21]:
fd = ['name','threesMade','FanDuel','ESPN BET','DraftKings','team','value','prob','fdBet']
espn = ['name','threesMade','ESPN BET','FanDuel','DraftKings','team','value','prob','espnBet']
dk = ['name','threesMade','DraftKings','team','value','prob','dkBet']

In [371]:
p =  .446 * .507 
nba.kellyCrit(p,412)* 62.5

2.392970873786409

In [938]:













df = pd.read_sql('''select name, abv_fgm + lc_fgm + rc_fgm as ThreesMade,abv_fga + lc_fga + rc_fga as ThreesAtts,
                   lc_fga + rc_fga as cornerAtts ,  lc_fgm + rc_fgm as cornerMakes
            from plyrLogs plogs 
            left join players p on plogs.player_id  = p.player_id
            where game_date = '{}' and name in ({})'''.format(yst,''''Tristan Da Silva','Toumani Camara' '''),nba.conn)
            

In [939]:
df

,name,ThreesMade,ThreesAtts,cornerAtts,cornerMakes
0,Tristan Da Silva,1,8,3,1
1,Toumani Camara,1,8,5,0


In [489]:
#run in batches of 500
batch = int(len(games) / 100)
missing = []
for i in range(0,batch):
    try:
        df = nba.get_plyr_drb_shots(games[i*100:i*100+100])
    except:
        missing.append(i)
    df.to_pickle('data/pickle/{}plyrDrb.pkl'.format(i))
    time.sleep(np.random.randint(20,45))
    

 52%|███████████████████████████████████████████████████████████████▍                                                          | 52/100 [04:27<05:57,  7.46s/it]/Users/ericcoxon/code/bets/nba/../bets.py:372: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  season_nullable = season).get_data_frames()[0][logCols]
 29%|███████████████████████████████████▍                                                                                      | 29/100 [03:47<09:12,  7.79s/it]/Users/ericcoxon/code/bets/nba/../bets.py:372: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior

In [619]:
plogs = pd.read_sql('select player_id,game_date, team_id from plyrLogs',nba.conn)

In [ ]:
BoxScoreAdvancedV3(

In [163]:
players = pd.read_sql('select player_id,name,min(game_date) as game_date from plyrLogs JOIN players using (player_id) group by player_id,name',nba.conn)

In [164]:
players[players.game_date>"2025-01-01"]

,player_id,name,game_date
236,1628467,Maxi Kleber,2025-01-06
324,1629029,Luka Doncic,2025-02-04
708,1631108,Max Christie,2025-02-04
812,1641740,Jaylen Clark,2025-01-09
816,1641747,DaRon Holmes II,2025-02-20
819,1641752,Bobi Klintman,2025-01-08
822,1641755,Kevin McCullar Jr.,2025-02-20
844,1641817,Anton Watson,2025-02-20
868,1642269,Devin Carter,2025-01-03
1042,203076,Anthony Davis,2025-02-04


In [2]:
from nba_api.stats.endpoints import BoxScoreAdvancedV3

In [ ]:
BoxScoreAdvancedV3(